In [ ]:
import json 
from pat import PatAnalysis, PatFlag

# Submit Job

In [ ]:
job_para = json.load(open('data/pat_job_test.json'))
job_para

# Run Sequential Code 

In [ ]:
%%time

job = PatAnalysis(job_para) 

job.extract_edm_rdm()
if job.check_data():
    if len(job.df_fac) > 0:
        job.net_of_fac()    

        job.allocate_with_psold()   
        job.df_pat.to_csv(r'C:\_Working\PAT_20201019\__temp\df_pat_svc.csv',index=False)
    

# Run with Service

In [3]:
import requests
import uuid
import json

# files = {'para': ('pat_para', open('data/pat_job_test.json','rt'))}

job_para = json.load(open('data/pat_job_test.json'))
job_para['job_guid'] = str(uuid.uuid4())

files = {'para': ('pat_para', json.dumps(job_para))}
ret = requests.post("http://localhost:5000/api/Jobs", files = files)
ret.status_code, ret.content

(200, b'Analysis submitted!')

# Results Summary

In [4]:
import requests
import pandas as pd

ret = requests.get("http://localhost:5000/api/Jobs")

pd.DataFrame(ret.json())

,job_id,job_guid,job_name,receive_time,update_time,status,user_name,user_email
0,100000047,97bd267e-d81f-45a8-b882-7dc7e4552e44 ...,PAT_Test_AIG,2021-09-28T03:14:05.740398,2021-09-28T03:15:19.752131,finished,cxiao,chengyou.xiao@guycarp.com
1,100000046,5be49de0-18d6-4402-b35f-ca97e49934dd ...,PAT_Test_AIG,2021-09-27T21:38:44.469166,2021-09-27T21:40:16.412275,finished,cxiao,chengyou.xiao@guycarp.com
2,100000043,8d0ac597-0fd7-4aa7-9e43-b6b8b0ebb76a ...,PAT_Test_AIG,2021-09-27T15:41:46.163242,2021-09-27T15:42:56.264525,finished,cxiao,chengyou.xiao@guycarp.com
